In [12]:
# modules we'll use
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

# read in all our data
HPP_data = pd.read_csv("../train.csv")

# set seed for reproducibility
np.random.seed(0)

In [13]:
#Impute values which are missing
HPP_data['LotFrontage'] = HPP_data['LotFrontage'].fillna('0')
HPP_data['Alley'] = HPP_data['Alley'].fillna('NoAlley')
HPP_data['BsmtQual'] = HPP_data['BsmtQual'].fillna('NoBasement')
HPP_data['BsmtCond'] = HPP_data['BsmtCond'].fillna('NoBasement')
HPP_data['BsmtExposure'] = HPP_data['BsmtExposure'].fillna('NoBasement')
HPP_data['BsmtFinType1'] = HPP_data['BsmtFinType1'].fillna('NoBasement')
HPP_data['BsmtFinType2'] = HPP_data['BsmtFinType2'].fillna('NoBasement')
HPP_data['FireplaceQu'] = HPP_data['FireplaceQu'].fillna('NoFireplace')
HPP_data['GarageType'] = HPP_data['GarageType'].fillna('NoGarage')
HPP_data['GarageYrBlt'] = HPP_data['GarageYrBlt'].fillna(-1)
HPP_data['GarageFinish'] = HPP_data['GarageFinish'].fillna('NoGarage')
HPP_data['GarageQual'] = HPP_data['GarageQual'].fillna('NoGarage')
HPP_data['GarageCond'] = HPP_data['GarageCond'].fillna('NoGarage')
HPP_data['PoolQC'] = HPP_data['PoolQC'].fillna('NoPool')
HPP_data['Fence'] = HPP_data['Fence'].fillna('NoFence')
HPP_data['MiscFeature'] = HPP_data['MiscFeature'].fillna('NoMiscFeature')

HPP_data['MasVnrArea'] = HPP_data['MasVnrArea'].fillna(HPP_data['MasVnrArea'].mean())
HPP_data['MasVnrType'] = HPP_data['MasVnrType'].fillna(HPP_data['MasVnrType'].mode()[random.randint(0, len(pd.Series(HPP_data['MasVnrType']).mode()) - 1)])
HPP_data['Electrical'] = HPP_data['Electrical'].fillna(HPP_data['Electrical'].mode()[random.randint(0, len(pd.Series(HPP_data['Electrical']).mode()) - 1)])



In [14]:
# Identify categorical columns
categorical_columns = HPP_data.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate through the categorical columns
for column in categorical_columns:
    # Convert all values to strings and then fit and transform the LabelEncoder
    HPP_data[column] = label_encoder.fit_transform(HPP_data[column].astype(str))

In [15]:
#baseline score for features
X = HPP_data.copy()
y = X.pop("SalePrice")

# Train and score baseline model
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

MAE Baseline Score: 1.776e+04


In [17]:
X = HPP_data.copy()
y = X.pop("SalePrice")

# Create synthetic features
X["AlleyPoolRatio"] = X['Alley'] / X["PoolQC"]

# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

MAE Score with Ratio Features: nan


/Users/frederickpretlove/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/frederickpretlove/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/frederickpretlove/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 327, in fit
    X, y = self._validate_data(
  File "/Users/frederickpretlove/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 581, in _vali